In [1]:
#import statements
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.enableHiveSupport().appName('Yelp').getOrCreate()
sc = spark.sparkContext

In [13]:
tip = spark.read.csv("gs://msca-bdp-student-gcs/group2/archive-3/yelp_academic_dataset_tip.csv",inferSchema=True, header=True)
user = spark.read.json("gs://msca-bdp-student-gcs/group2/archive-3/yelp_academic_dataset_user.json")
business = spark.read.csv("gs://msca-bdp-student-gcs/group2/archive-3/yelp_academic_dataset_business.json",inferSchema=True, header=True) 
checkin = spark.read.csv("gs://msca-bdp-student-gcs/group2/archive-3/yelp_academic_dataset_checkin.csv",inferSchema=True, header=True)
review = spark.read.json("gs://msca-bdp-student-gcs/group2/archive-3/yelp_academic_dataset_review.json")

In [14]:
# friends column shows all userIDs that are friends of a user. 
# we don't need that and it blocks visibility of other cols.

user = user.drop('friends')
user.cache()


23/08/03 01:38:45 WARN org.apache.spark.sql.execution.CacheManager: Asked to cache already cached data.


DataFrame[average_stars: double, compliment_cool: bigint, compliment_cute: bigint, compliment_funny: bigint, compliment_hot: bigint, compliment_list: bigint, compliment_more: bigint, compliment_note: bigint, compliment_photos: bigint, compliment_plain: bigint, compliment_profile: bigint, compliment_writer: bigint, cool: bigint, elite: string, fans: bigint, funny: bigint, name: string, review_count: bigint, useful: bigint, user_id: string, yelping_since: string]

In [16]:
review.take(1)

[Row(business_id='XQfwVwDr-v0ZS3_CbbE5Xw', cool=0, date='2018-07-07 22:09:11', funny=0, review_id='KU_O5udG6zpxOg-VcAEodg', stars=3.0, text="If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.", useful=0, user_id='mh_-eMZ6K5RLWhZyISBhwA')]

In [17]:
user.take(1)

[Row(average_stars=3.91, compliment_cool=467, compliment_cute=56, compliment_funny=467, compliment_hot=250, compliment_list=18, compliment_more=65, compliment_note=232, compliment_photos=180, compliment_plain=844, compliment_profile=55, compliment_writer=239, cool=5994, elite='2007', fans=267, funny=1259, name='Walker', review_count=585, useful=7217, user_id='qVc8ODYU5SZjKXVBgXdI7w', yelping_since='2007-01-25 16:47:26')]

In [7]:
review.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [8]:
user.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)



In [18]:
tip.take(1)

[Row(_c0='0', user_id='AGNUgVwnZUey3gcPCJ76iw', business_id='3uLgwr0qeCNMjKenHJwPGQ', text='Avengers time with the ladies.', date='2012-05-18 02:17:21', compliment_count='0')]

In [19]:
business.take(1)

[Row(_c0='0', user_id='AGNUgVwnZUey3gcPCJ76iw', business_id='3uLgwr0qeCNMjKenHJwPGQ', text='Avengers time with the ladies.', date='2012-05-18 02:17:21', compliment_count='0')]

In [21]:
checkin.take(4)

[Row(_c0=0, business_id='---kPU91CF4Lq2-WlRu9Lw', date='2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020-07-24 22:42:27, 2020-10-24 21:36:13, 2020-12-09 21:23:33, 2021-01-20 17:34:57, 2021-04-30 21:02:03, 2021-05-25 21:16:54, 2021-08-06 21:08:08, 2021-10-02 15:15:42, 2021-11-11 16:23:50'),
 Row(_c0=1, business_id='--0iUa4sNDFiZFrAdIWhZQ', date='2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011-07-18 22:30:31, 2012-09-07 20:28:50, 2013-03-27 15:57:36, 2013-08-13 00:31:34, 2013-08-13 00:31:48, 2013-09-23 17:39:38, 2013-11-18 06:34:08, 2014-04-12 23:04:47'),
 Row(_c0=2, business_id='--30_8IhuyMHbSOcNWd6DQ', date='2013-06-14 23:29:17, 2014-08-13 23:20:22'),
 Row(_c0=3, business_id='--7PUidqRWpRSpXebiyxTg', date='2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012-03-11 10:30:02, 2012-04-24 07:07:59, 2012-04-24 07:43:31, 2013-05-25 16:41:10, 2014-05-02 15:49:55, 2014-09-18 02:28:23, 2014-11-10 15:16:43, 2015-09-27 13:18:32')]

In [9]:
# spark.stop()